In [1]:
import requests
from bs4 import BeautifulSoup
import time
import re
import datetime
import telebot


In [2]:
TIMEOUT = 60 * 5
TOKEN = ""
path = "https://api.telegram.org/bot" + TOKEN + "/"
VKpath = "https://api.vk.com/method/"
access_token = ""
chatID = 
v = "5.122"

In [7]:
bot = telebot.TeleBot(TOKEN, parse_mode=None)

In [3]:
def log(text):
    print("[{}] {}".format(datetime.datetime.now().strftime("%X"), text))


class Meme:
    def __init__(self, json_meme):
        self.date = json_meme['date']
        self.text = json_meme['text']
        if 'copy_history' in json_meme:
            json_meme = json_meme['copy_history'][0]
            if self.text != "":
                self.text += "\n"
            self.text += json_meme['text']
        self.photos = []
        if 'attachments' in json_meme:
            for json_photo in json_meme['attachments']:
                if json_photo['type'] == 'photo':
                    self.photos.append(json_photo['photo']['sizes'][-1]['url'])

    def send(self, chat_id):
        if len(self.photos) == 0:
            bot.send_message(chat_id=chat_id, text=self.text)
        if len(self.photos) == 1:
            bot.send_photo(chat_id=chat_id, photo=self.photos[0], caption=self.text)
        if len(self.photos) > 1:
            media = []
            for photo in self.photos:
                media.append(telebot.types.InputMediaPhoto(photo))
            media[0].caption = self.text
            bot.send_media_group(chat_id=chat_id, media=media)

    def __lt__(self, other):
        return self.date < other.date


def wall_get(group_id, count=50):
    resp = requests.get(VKpath+"wall.get", {'owner_id': -group_id,
                                            'count': count,
                                            'filter': 'owner',
                                            'access_token': access_token,
                                            'v': v})
    if resp.status_code != 200:
        return []
    meme_list = []
    for obj in resp.json()['response']['items']:
        meme_list.append(Meme(obj))
    meme_list.sort()
    return meme_list

In [8]:
ex = wall_get(54732939, 2)
ex[-1].send(chatID)

ApiException: A request to the Telegram API was unsuccessful. The server returned HTTP 400 Bad Request. Response body:
[b'{"ok":false,"error_code":400,"description":"Bad Request: group send failed"}']

In [79]:
class InputMediaPhoto:
    def __init__(self, media, caption=None):
        self.type = 'photo'
        self.media = media
        self.caption = None
        
    def to_json(self):
        res_full = {'type': self.type,
                'media': self.media,
                'caption': self.caption}
        res = {}
        for key in res_full.keys():
            if res_full[key] is not None:
                res[key] = res_full[key]
        return res


class Update:
    def __init__(self, json_obj):
        self.update_id = json_obj['update_id']
        self.chat_id = json_obj['message']['chat']['id']
        if 'text' in json_obj['message']:
            self.text = json_obj['message']['text']
        else:
            self.text = ""
            

class Telebot:
    def __init__(self, token, offset=None, timeout=None):
        self.token = token
        self.offset = offset
        self.timeout = timeout
        
    def make_request(self, method_name, params):
        path = "https://api.telegram.org/bot" + self.token + "/" + method_name
        new_params = {}
        for key in params.keys():
            if params[key] is not None:
                new_params[key] = params[key]
        params = new_params
        resp = requests.get(path, json=params)
        if resp.status_code != 200:
            log(resp.json())
            raise ConnectionError("Не удалось получить ответ от сервера.")
        if resp.json()['ok'] == False:
            raise ConnectionError("Неправильный запрос.")
        return resp.json()['result']
    
    def send_message(self, chat_id, text):
        method_name = r"sendMessage"
        params = {'chat_id': chat_id,
                  'text': text}
        self.make_request(method_name, params)
        
    def send_photo(self, chat_id, photoURL):
        method_name = r"sendPhoto"
        params = {'chat_id': chat_id,
                  'photo': photoURL}
        self.make_request(method_name, params)
        
    def send_media_group(self, chat_id, media):
        method_name = r"sendMediaGroup"
        media = list(map(InputMediaPhoto.to_json, media))
        params = {'chat_id': chat_id,
                  'media': media}
        self.make_request(method_name, params)
        
    def get_updates(self, offset=None, timeout=None, allowed_updates=['message']):
        method_name = r"getUpdates"
        if offset is None:
            offset = self.offset
        if timeout is None:
            timeout = self.timeout
        params = {'offset': offset,
                  'timeout': timeout,
                  'allowed_updates': allowed_updates}
        updates = self.make_request(method_name, params)
        updates = list(map(Update, updates))
        if len(updates) != 0:
            self.offset = updates[-1].update_id+1
        return updates

In [80]:
bot = Telebot(TOKEN)
# bot.send_message(chatID-1, "Hi")

In [85]:
photo = InputMediaPhoto("https://sun1-18.userapi.com/B_liWIvHQSlWZc-E2odXW4Hxmv9GqNXqeL38qg/gp1aRzUyTlA.jpg")
photoURL = "https://external-content.duckduckgo.com/iu/?u=http%3A%2F%2F4.bp.blogspot.com%2F-Jx21kNqFSTU%2FUXemtqPhZCI%2FAAAAAAAAh74%2FBMGSzpU6F48%2Fs1600%2Ffunny-cat-pictures-047-001.jpg&f=1&nofb=1"
bot.send_media_group(chatID, [photo])
# resp = requests.get(path+"sendMediaGroup", json={'chat_id': chatID,
#                                                 'media': [photo.to_json(), photo.to_json()]})
# resp.json()
# photo.to_json()
# bot.get_updates(timeout=100)

TypeError: 'InputMediaPhoto' object is not iterable

In [3]:
resp = requests.get(VKpath+"wall.get", {'owner_id': -157081760,
                                            'count': 3,
                                            'filter': 'owner',
                                            'access_token': access_token,
                                            'v': v})
resp.json()

{'error': {'error_code': 29,
  'error_msg': 'Rate limit reached',
  'request_params': [{'key': 'owner_id', 'value': '-157081760'},
   {'key': 'count', 'value': '3'},
   {'key': 'filter', 'value': 'owner'},
   {'key': 'v', 'value': '5.122'},
   {'key': 'method', 'value': 'wall.get'},
   {'key': 'oauth', 'value': '1'}]}}